This notebook is created to compare two sources: list of Chargers and list of Terminals. Goal of it is to add data to file where terminals data is kept and to find terminals without a match.

In [ ]:
import configparser
import mariadb
import pandas as pd

config = configparser.ConfigParser()
config.read(r'\config.ini')

try:
    connection = mariadb.connect(
        user=config['database']['username'],
        password=config['database']['password'],
        host=config['database']['host'],
        port=int(config['database']['port']),
        database='db_name'
    )
    cursor = connection.cursor()
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")


In [ ]:
query = """
    SELECT *
    FROM ChargePointConfigsBig
    """
cursor.execute(query)
result = cursor.fetchall()

In [ ]:
df_chargers = pd.DataFrame(result, columns=[col[0] for col in cursor.description])
df_chargers

In [ ]:
connection.close()

In [ ]:
df_terminal = pd.read_excel(r"\terminals.xlsx")
df_terminal

In [ ]:
df_terminal['Charger Serial No.'] = df_terminal['Charger Serial No.'].astype('str')

In [ ]:
df_terminal.info()

In [ ]:
df_terminal2 = df_terminal.copy()

In [ ]:
df_terminal2 = df_terminal2.merge(df_chargers[['SerialNumber', 'ConfigCleansed', 'LocationFriendlyName', 'FriendlyCode']], left_on='Charger Serial No.', right_on='SerialNumber', how='left').reset_index(drop=True)
df_terminal2.drop(columns=['SerialNumber'], inplace=True)

df_terminal2.info()

In [ ]:
df_terminal3 = df_terminal2.loc[df_terminal2['ConfigCleansed'].isna()]
df_terminal3.info()

In [ ]:
df_terminal3.to_excel('terminal_empties.xlsx', index=False)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified too
    print(df_terminal2) 